# TensorFlow模型的跨平台部署

## 背景

由于公司由于业务发展需要，结合公司现有技术栈、性能等因素考虑，现需要将在Python投研环境通过TensorFlow训练的模型移植到C++实盘生产环境，连调研带采坑实现前前后后花了三四天，先将整个过程与遇到的坑整理为笔记，方便以后查阅。

## 可选方案

事实上部署通过TensorFlow训练完成的模型方案有很多，仅仅是官网就提供了三种方案可选。

- 使用其他语言的TensorFlow    
包括C、C++、Java、Go、Swift等等。但是需要注意的是，存在一些令人困惑的现象，例如，如果你希望用C语言部署你的模型，那么你可能不用花很大的精力在编译TensorFlow在C中的动态链接库，如果你使用OSX，你甚至可以直接通过`brew install libtensorflow`一行命令完成动态链接库的安装与环境变量的导出。但是，官网没有提供任何C语言版本TensorFlow的文档，你只能通过动态库的头文件简短的注释去揣测每个API使用方法。而另一方面，使用C++版本的TensorFlow你需要手动从源代码尝试编译C++的动态库，这会遇到很多奇怪的问题，尤其当你是OSX用户时，这点之后的笔记会详细展开。由于Java、Go、Swift等没有尝试，这里就不再赘述了。

- 使用saved_model_cli
官网提供了较为详细的关于这个命令行工具的使用方法，大致是，如果你的系统Python环境，或者虚拟Python环境已经安装了TensorFlow，那么你可以通过这个工具轻而易举地从通过`SavedModelBuilder`这个模块保存的模型恢复并使用它在生产环境下工作，它同时支持通过命令行参数加载`.npy`类型的numpy数组直接作为模型的输入。由于官网有很详细的中文文档，所以这里也就不再赘述了。

- 使用TensorFlow Serving
Serving是一个TensorFlow专门用来做模型部署的模块，但是目前官网提供的文档还是英文的，在写这篇笔记时我还没有看完，按照计划应该是明天调研完毕。它支持将模型服务化，像一个Web服务一样运行在某个端口，然后通过gRPC与遵从RESTful-API的接口直接调用模型的预测功能，但是我还没有看完，希望明天能将这部分补全。

## 方案一：使用其他语言的TensorFlow

### 从源代码编译TensorFlow的C++动态链接库

首先，需要从GitHub上的 [TensorFlow](https://github.com/tensorflow/tensorflow) 仓库Clone源代码到目标机器，命令：
```
git clone git@github.com:tensorflow/tensorflow.git
```

#### 对应于 Ubuntu 16.04

然后，请确保以下依赖的对应版本已经正确安装：
- eigen >= 3.3.3   
  对于eigen，推荐通过源码安装，首先下载源码压缩包并解压：
  ```
  wget http://bitbucket.org/eigen/eigen/get/3.3.4.zip
  ```
  然后通过cmake生成MakeFile，注意eigen不允许In-Source builds，需要创建一个文件夹，可以起名为build，然后运行cmake命令：
  ```
  mkdir build && cd build
  cmake .. && make
  make install
  ```
- protobuf == 3.6.0    
  对于Protobuf的版本是3.6.0，是在
  ```
  tensorflow-master/tensorflow/contrib/cmake/external/protobuf.cmake
  ```
  这个目录指定的，如果没有这个文件，请先运行`cmake .`指令。
  对于protobuf的安装依然是推荐源码安装，安装过程基本同上：
  ```
  wget https://github.com/google/protobuf/releases/download/v3.6.0/protobuf-all-3.6.0.zip
  ```
  然后一次运行配置，编译、安装。
  ```
  ./configure
  make
  make install
  ```

当然，在当前系统Python环境下，或者虚拟Python环境下的依赖也是必须的。
```
sudo apt-get install python-numpy python-dev python-pip python-wheel
```
如果以上指令出现权限问题或者环境变量未导出请自行考虑解决。

#### 对应于 OSX 10.13.6

流程完全相同，有几处不同如下：
- eigen的安装请通过以下指令安装：
```
brew install --HEAD eigen
```
经过多次采坑和StackOverflow，确认是3.3.3和3.3.4版本的eigen存在一个C++模板偏特化的Bug，没有没有在Release版本中解决，请安装以上命令安装eigen，可以节省你一天时间。

当然，在当前系统Python环境下，或者虚拟Python环境下的依赖也是必须的。
```
pip install six numpy wheel 
brew install coreutils
```
如果以上指令出现权限问题或者环境变量未导出请自行考虑解决。

#### 安装 Bazel

这里由于Bazel官网有较为详细的教程，所以就不在此赘述了，请参阅：   
- [Ubuntu 16.04 Bazel 安装教程](https://docs.bazel.build/versions/master/install-ubuntu.html)     
- [OSX Bazel 安装教程](https://docs.bazel.build/versions/master/install-os-x.html)    

#### 运行configure

首先进入刚才TensorFlow的源码目录，执行以下命令：
```
./configure
```
然后根据交互式引导选择需要启用的编译选项，如是否启用CUDA等等。

#### 编译C++动态库

请执行以下指令编译TensorFlow的C++动态库：
```
bazel build //tensorflow:libtensorflow_cc.so
```
编译完成后，务必执行一下操作：
```
cp -r bazel-genfiles/ /usr/local/include/
cp -r tensorflow /usr/local/include/
cp -r third_party /usr/local/include/
cp -r bazel-bin/libtensorflow_cc.so /usr/local/lib/
cp -r bazel-bin/libtensorflow_framework.so /usr/local/lib/
```
至此，C++版本的TensorFlow动态库就编译完成了。

#### 验证动态链接库

可以新建一个C++项目，通过官网C++的API文档的一个 [简单例子](https://www.tensorflow.org/api_guides/cc/guide) 验证C++版本的TensorFlow是否正常运行。
```
#include "tensorflow/cc/client/client_session.h"
#include "tensorflow/cc/ops/standard_ops.h"
#include "tensorflow/core/framework/tensor.h"

int main() {
  using namespace tensorflow;
  using namespace tensorflow::ops;
  Scope root = Scope::NewRootScope();
  // Matrix A = [3 2; -1 0]
  auto A = Const(root, { {3.f, 2.f}, {-1.f, 0.f} });
  // Vector b = [3 5]
  auto b = Const(root, { {3.f, 5.f} });
  // v = Ab^T
  auto v = MatMul(root.WithOpName("v"), A, b, MatMul::TransposeB(true));
  std::vector<Tensor> outputs;
  ClientSession session(root);
  // Run and fetch v
  TF_CHECK_OK(session.Run({v}, &outputs));
  // Expect outputs[0] == [19; -3]
  LOG(INFO) << outputs[0].matrix<float>();
  return 0;
}
```
如果程序运行正常，那么结果会输出19和-3。

### 使用saved_model_builder进行模型持久化

#### 为什么不使用checkpoint？

`Saver.restore()`需要提前建立好计算图，这在理论上是可行的，但是对于模型跨平台来说，成本和效率都存在问题，当模型趋于复杂，序列模型、深度卷积、复杂全连接以及种种超参数以及优化技术都需要两端完全匹配，就目前来看是得不偿失的。

#### 使用saved_model_builder持久化模型

我们采用`saved_model_builder`持久化模型，这种方式持久化的模型会将计算图结构及其参数持久化为Protobuf文件到磁盘，这种方法是可以直接跨平台，跨语言恢复模型的。本质上，`saved_model_builder`持久化模型的过程即是将计算图结构，及其描述信息，以及变量名及其值等一系列模型相关的信息序列化为Protobuf的过程，在这个过程中，我们首先需要构造一个`signature_def`，用来指导并标记一些关键计算图节点或者边缘的序列化过程，我们直接看代码：
```
session = tf.Session()

x_input = tf.placeholder(tf.float32, [None, 1])
y_input = tf.placeholder(tf.float32, [None, 1])

fc1 = tf.layers.dense(x_input, 10, tf.nn.relu)
fc2 = tf.layers.dense(fc1, 10, tf.nn.relu)

y_predict = tf.layers.dense(fc2, 1)

loss_func = tf.losses.mean_squared_error(labels=y_input, predictions=y_predict)

optimizer = tf.train.AdamOptimizer().minimize(loss_func)

session.run(tf.global_variables_initializer())
```

#### 构造`signature_def`对象

这段代码初始化了一个Session，并计算图的结点和边缘信息，直觉地，这个简单的双层感知机模型用来做一个简单的回归问题，接下来，我们需要根据这个已经构造好的计算图调用如下API构造`signature_def`对象：
```
signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={
        'x_input': tf.saved_model.utils.build_tensor_info(x_input),
        'y_input': tf.saved_model.utils.build_tensor_info(y_input)
    },
    outputs={
        'y_predict': tf.saved_model.utils.build_tensor_info(y_predict),
        'loss_func': tf.saved_model.utils.build_tensor_info(loss_func)
    },
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)
```
可以看到，这个API接收三个重要参数，其中`inputs`即是`tf.placeholder`，用于描述输入张量，`outputs`即是输出张量。在这里，可以看到我们分别用字符串`x_input`与`y_input`，`y_predict`与`loss_func`作为序列化后获取张量的键，而`tf.saved_model.utils.build_tensor_info`就是将张量转为protobuff结构的快捷方法。总结一下，我们用`tf.saved_model.signature_def_utils.build_signature_def`方法构造了`signature_def`对象，这个对象包含了计算图中输入与输出张量的键值对信息，键即是张量名，值即是protobuff结构的张量，用一个`method_name`键来描述功能。

#### 模型持久化

我们还是直接看代码：
```
for step in range(2000):
    session.run(optimizer, {
        x_input: x_train,
        y_input: y_train
    })

    if (step + 1) % 500 == 0:    
        if os.path.exists(graph_save_dir):
            shutil.rmtree(graph_save_dir)
        builder = tf.saved_model.builder.SavedModelBuilder(graph_save_dir)
        builder.add_meta_graph_and_variables(session,
                                             [tf.saved_model.tag_constants.SERVING],
                                             {tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature})
        builder.save()
```
这段代码一边进行训练，一边检查是否需要持久化模型，这里有两个坑：
- 我们需要在每一次需要持久化模型时创建一个`SavedModelBuilder`，而不能在训练全局仅仅初始化一个`SavedModelBuilder`，由于某些原因，如果你这么做，你会在第二次`add_meta_graph_and_variables`时捕获到异常，大意是说计算图的元信息和变量已经添加到`SavedModelBuilder`了，不可以重复添加，而`add_meta_graph`也是在做相同的事情。   
- 你需要在每一次需要持久化模型时清空模型持久化目录，如果你不这么做，你会在`builder.save()`时捕获异常，大意是提示持久化路径不为空，不可以写入，你需要手动清空，事实上，在后文我们介绍Serving时会提到，TensorFlow这么做的设计是希望我们有一个版本号文件夹描述模型持久化版本，但是出于某些原因，这个设计与我们的模型持久化规有冲突，例如，我们希望根据验证集的准确率变化来决定是否执行Early Stop，以及更新持久化模型，但是我们又不认为这时候该更新版本号等等。    
然后来简单介绍一下`add_meta_graph_and_variables`这个API，这个API的上一行是实例化`SavedModelBuilder`，这个就不展开了，需要给定一个持久化目录。对于`add_meta_graph_and_variables`主要接受三个参数，第一个即是Session对象，它包含了计算图和变量的信息，第二个变量是一个Tags数组，我们可以根据需求来设定Tag，例如这里的例子即是设定一个用于Serving的模型，第三个参数非常重要，他是一个字典对象，它的一个重要键值对即是：
```
{tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature}
```
即`default_serving`与`signature_def`对象，顾名思义，这个键值对描述了在持久化时和恢复时我们如何找到计算图中的输入和输出节点。
最后，万事俱备了，我们只需要调用`builder.save()`，来持久化我们的模型。

#### Python环境下的模型恢复

直接看代码：
```
# Session.
session = tf.Session()

# Load meta graph.
meta_graph_def = tf.saved_model.loader.load(session, [tf.saved_model.tag_constants.SERVING], graph_save_dir)  # type: tf.MetaGraphDef

# Get signature.
signature_def = meta_graph_def.signature_def
signature = signature_def[tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

# Get input tensor.
x_input_tensor = signature.inputs['x_input'].name
y_input_tensor = signature.inputs['y_input'].name

# Get output tensor.
y_predict_tensor = signature.outputs['y_predict'].name

# Get loss func.
loss_op = signature.outputs['loss_func'].name

_, loss = session.run([y_predict_tensor, loss_op], {
    x_input_tensor: x_train,
    y_input_tensor: y_train,
})

logging.warning('Loss: {}'.format(loss))
```
这段代码首先初始化了一个Session对象，然后通过`tf.saved_model.loader.load`API加载`meta_graph_def`对象，这个API接受三个参数，第一个是Session，第二个是在持久化模型时设定的Tags，第三个是模型的持久化路径。`meta_graph_def`对象是一个已经被反序列化的protobuff对象，我们可以轻易地通过点语法来获取`signature_def`对象。`signature_def`对象已经在上文提到，这个对象包含了计算图中输入与输出张量的键值对信息，键即是张量名，值即是protobuff结构的张量，用一个`method_name`键来描述功能。这个对象在持久化时由键值对：
```
{tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature})
```
来确定，而获取了`signature_def`对象，我们就可以获取在持久化前通过键值对定义的关键张量了，而正如代码所展示的，获取了关键张量，我们就可以提供输入，来通过Session计算模型预测结果，到此，python环境下的模型恢复就结束了。

#### C++环境下的模型恢复

依然是直接看代码：
```
int main() {
    using namespace tensorflow;
    using namespace tensorflow::ops;

    SessionOptions sessionOptions;
    RunOptions runOptions;

    SavedModelBundle bundle;
    Status status;

    status = LoadSavedModel(sessionOptions, runOptions, GraphDir, {kSavedModelTagServe}, &bundle);

    if (!status.ok()) {
        std::cerr << "Load model failed, reason: " << status.ToString() << std::endl;
        return -1;
    }

    Scope root = Scope::NewRootScope();
    std::vector<Tensor> yPredict;

    auto signature = bundle.meta_graph_def.signature_def().find("signature")->second;
    auto inputs = signature.inputs();
    auto outputs = signature.outputs();

    auto xInputTensor = inputs.find("x_input")->second;
    auto yPredictTensor = outputs.find("y_predict")->second;

    std::vector<double > x(X, X + 640);

    Tensor input(DT_FLOAT, TensorShape({1, 640}));

    std::copy_n(x.begin(), x.size(), input.flat<float>().data());

    TF_CHECK_OK(bundle.session->Run({{xInputTensor.name(), input}}, {yPredictTensor.name()}, {}, &yPredict));

    LOG(INFO) << yPredict[0].matrix<float>();

    return 0;
}
```
对于C++的代码就不做过多解释了，原理大致与Python类似，最大的不同是C++不直接初始化一个Session，而是初始化一个`SavedModelBundle`引用，然后调用`LoadSavedModel`传`SavedModelBundle`引用于Tags信息恢复模型，恢复的模型Session将会是`SavedModelBundle`的一个成员变量。之后的套路也与python大致相同，即获取`signature_def`对象，然后通过持久化前定义的键值对获取关键张量，然后提供输入给Session就可以计算结果。至此C++的模型恢复也就到此结束了。而事实上这个过程是及其艰辛的，C++的TensorFlow的文档不尽完善，甚至找不到通过恢复模型而得到的`Session`对象如何使用，而官网也只有`ClientSession`这样更高级的对象的介绍，以及诸多关于protobuff在C++中的使用问题，当然更多的原因还是笔者的C++太菜，还希望各位多多包涵。